In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg

data:
  iters_per_epoch: '?' 
  train_splits: cal
  val_splits: val 

dataloader:
  batch_size: '?'
  num_workers: 1
  pin_memory: True 

optim: # Unclear if we should tune this or not.
  _class: torch.optim.Adam
  lr: '?'

train:
  epochs: '?'
  eval_freq: '?'
  base_pretrained_dir: '?'
  base_checkpoint: 'max-val-dice_score'
  base_pt_select_metric: 'val-dice_score'

loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml log_cfg

log:
  checkpoint_freq: 50
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True
    ece_loss:
      _fn: ese.metrics.image_ece_loss
      num_prob_bins: 15
      from_logits: True
    area_estimation_error:
      _fn: ese.losses.area_estimation_error
      from_logits: True
      square_diff: False

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: 
          - ece_loss 
          - area_estimation_error
        phase: val

<IPython.core.display.Javascript object>

In [5]:
%%yaml experiment_cfg 

name: "ISLES_3D_TS"
# name: "ISLES_3D_LTS"

train: 
    base_pretrained_dir: 
        - "/storage/vbutoi/scratch/ESE/training/09_08_24_ISLES_3D_CrossEntropy"
        # - "/storage/vbutoi/scratch/ESE/training/09_08_24_ISLES_3D_Dice"

data:
    _class: ese.datasets.ISLES

model:
    - TempScaling
    # - LTS 

dataloader:
  batch_size: 4 

optim:
  lr: 1.0e-5 


loss_func:
    _class: ese.losses.AreaEstimationError

<IPython.core.display.Javascript object>

In [6]:
from ese.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([log_cfg, callbacks_cfg])

updated_base_cfg, cal_cfgs = get_ese_calibration_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg,
    calibration_model_cfgs={}
)

In [7]:
len(cal_cfgs)

4

## Running Jobs

In [8]:
from ese.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
####### Run individual jobs
run_ese_exp(
    config=cal_cfgs[0], 
    experiment_class=PostHocExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='1',
    # gpu='4',
)

Intel MKL extensions not available for NumPy

Using slow Pillow instead of Pillow-SIMD

Set seed: 40


Intel MKL extensions not available for NumPy

Using slow Pillow instead of Pillow-SIMD

Loading checkpoint: max-val-dice_score.
Running PostHocExperiment("/storage/vbutoi/scratch/ESE/calibration/debug/20240909_150021-8OD7-2d513d258a3f99053d50f1515c1aaf50")
---
callbacks:
  epoch:
  - ionpy.callbacks.ETA
  - ionpy.callbacks.JobProgress
  - ionpy.callbacks.TerminateOnNaN
  - ionpy.callbacks.PrintLogged
  - ionpy.callbacks.ModelCheckpoint:
      monitor:
      - ece_loss
      - area_estimation_error
      phase: val
  step:
  - ese.callbacks.ShowPredictions
data:
  iters_per_epoch: 1000
  preload: false
  version: 1.0
dataloader:
  batch_size: 4
  num_workers: 1
  pin_memory: true
experiment:
  seed: 40
log:
  checkpoint_freq: 50
  metrics:
    area_estimation_error:
      _fn: ese.losses.area_estimation_error
      from_logits: true
      square_diff: false
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: mean
      from_logits: true
      ignore_empty_labels: false
    ece_loss:
      _fn: ese.metrics.image_ece_loss
      from_logits: true
      

AssertionError: y_pred must have 4 dimensions, got torch.Size([4, 1, 148, 180, 71])

In [ ]:
#### Run Batch Jobs
submit_ese_exps(
    group="calibration",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=cal_cfgs,
    experiment_class=PostHocExperiment,
    track_wandb=True,
    available_gpus=['0', '1', '2', '3']
    # available_gpus=['4', '5', '6', '7']
)